# Deep Video Analytics tutorial

### Please run this notebook on Host OS or Google cloud sheel with dvactl configured & DVA running

In [1]:
import os, requests, sys, json, jsonschema

### This tutorial uses REST API endpoint using a DVA python client.

In [14]:
sys.path.append('../../client/') #add DVA client to python path 

In [3]:
import dvaclient
import logging, json

## Using Deep Video Analytics python client

####  Following command generate & copy a REST API token, it can also be found at /security page from the sidebar in DVA UI

In [4]:
creds = json.load(file('../../creds.json'))
server, token = creds['server'], creds['token']
ctx = dvaclient.context.DVAContext(server=server,token=token)

## REST API vs DVAPQL

Deep Video Analytics uses REST API (GET, LIST, but **NOT POST**) for exposing state of the system along metadata and urls to raw contents such as videos and images. **/queries is the only endpoint that supports POST requests** for mutating the state of the system. The user submits a DVAPQL query for loading videos/datasets, launching processing jobs, importing models and performing visual search. 

### List videos

In [15]:
for v in ctx.list_videos()['results']:
    print "{name} with ID: {vid}".format(name=v['name'],vid=v['id'])

coco_train_ci with ID: b1eb4434-1fdc-4775-9d51-1fe122d0feab
exampledataset with ID: 1aa434ed-4e05-4bbc-bc2b-e3708fbd1efb
crnn_test with ID: 9b770b88-dad0-4ca7-ac33-dae627176e6e
video_map_reduce with ID: a9a4c3ef-a591-4c3e-895c-fa5cb88d2972
citest with ID: 801d50ab-32ad-410b-a93d-58e75cc09185
spectre with ID: e61b3209-8cc7-48d7-a4b6-75cb10c0b79e


### List configured indexing, detection and analysis models

In [16]:
for v in ctx.list_models()['results']:
    print v['name'], v['model_type']

coco D
textbox D
face D
inception I
facenet I
crnn A
tagger A
Facenet_LOPQ_on_LFW P
Inception_PCA_from_Youtube8M P


## List events

In [ ]:
_ = ctx.list_events(verbose=True)

## Process a video

In [6]:
video_processing_query = dvaclient.query.ProcessVideoURL(name="spectre",url="https://www.youtube.com/watch?v=ashLaclKCik")
video_processing_query.execute(ctx)

##### Optionally you can "wait" on it to complete,

In [7]:
video_processing_query.wait(verbose=True)

KeyError: 'map'

## Finding similar images

### Query image

In [8]:
leo_query_image = 'figures/query_leo.png'

### Get configured retrievers and indexers

In [12]:
indexers = {r['name']:r for r in ctx.list_models()[u'results'] if r['model_type'] == 'I'}
retrievers = {r['name']:r for r in ctx.list_retrievers()[u'results']}

### Make a query using inception retriever and inception indexer

In [13]:
inception_indexer_pk = indexers['inception']['id']
inception_retriever_pk = retrievers['inception']['id']
q = dvaclient.query.FindSimilarImages(leo_query_image,indexer_pk=inception_indexer_pk,retriever_pk=inception_retriever_pk)
q.execute(context=ctx)
q.wait()
q.gather_search_results()

KeyError: 'map'

### Visualize results

In [ ]:
for r in q.search_results:
    r.visualize()

In [ ]:
q = dvaclient.query.FindSimilarImages('figures/query_car.png',indexer_pk=inception_indexer_pk,retriever_pk=inception_retriever_pk)
q.execute(context=ctx)
q.wait()
q.gather_search_results()
for r in q.search_results:
    r.visualize()